In [2]:
config_excel_path="abfss://enterpriseanalytics@fedsaentanalytics.dfs.core.windows.net/config/smoke_test_transform_config.xlsx"

In [3]:
import pandas as pd

config_path="abfss://enterpriseanalytics@fedsaentanalytics.dfs.core.windows.net/config/fe_config_transform/"
config_refs_path="abfss://enterpriseanalytics@fedsaentanalytics.dfs.core.windows.net/config/fe_config_transform_ref/"

config_df = pd.read_excel(config_excel_path, sheet_name=0)
source_df_config = spark.createDataFrame(config_df)
source_df_config.createOrReplaceTempView("tmp_config")

config_refs_df = pd.read_excel(config_excel_path, sheet_name=1)
source_df_config_refs = spark.createDataFrame(config_refs_df)
source_df_config_refs.createOrReplaceTempView("tmp_config_refs")

display(source_df_config)
display(source_df_config_refs)

In [4]:
# do checks
configs = spark.sql("""select lower(array_join(collect_list(distinct concat('\\'',config_name,'\\'')), ',')) configs
 from tmp_config
""").first().configs

print(configs)

ref_az_ids = spark.sql(f"""select lower(array_join(collect_list(distinct concat('\\'',az_id,'\\'')), ',')) configs
 from (select distinct az_id from tmp_config 
            union select distinct az_id from tmp_config_refs
            union select distinct az_id from vw_fe_config_transform_ref where config_name in ({configs})
    ) x 
""").first().configs

print(ref_az_ids)

if spark.sql("select az_id from tmp_config group by az_id having count(*) > 1").count() > 0:
    raise Exception(f"There are duplicate az_ids in {config_excel_path}")

if spark.sql("select az_id,parent_az_id from tmp_config_refs group by az_id,parent_az_id having count(*) > 1").count() > 0:
    raise Exception(f"There are duplicate az_id refs in {config_excel_path}")

if spark.sql("select * from tmp_config").count() <= 0:
    raise Exception(f"There are no rows returned from {config_excel_path}")

if spark.sql("select * from tmp_config_refs").count() <= 0:
    raise Warning(f"There are no rows returned for refs from {config_excel_path}")

In [5]:
spark.sql(f"delete from delta.`{config_path}` t where lower(config_name) in ({configs})")

spark.sql(f"""
insert into delta.`{config_path}` (uuid, config_name, dest_delta_container, dest_delta_table_path, dest_delta_schema_name, dest_delta_table_name, dest_delta_table_primary_key, dest_delta_table_partition_by, dest_delta_table_z_order, sql_transform_init, sql_transform_inc, src_col_def, tag, mode, dest_dw_endpoint, dest_dw_db, is_disabled)
    select uuid() uuid, config_name, dest_delta_container, dest_delta_table_path, dest_delta_schema_name, dest_delta_table_name, dest_delta_table_primary_key, dest_delta_table_partition_by, dest_delta_table_z_order, sql_transform_init, sql_transform_inc, src_col_def, tag, mode, dest_dw_endpoint, dest_dw_db, is_disabled
    from tmp_config
""")

In [6]:
spark.sql(f"delete from delta.`{config_refs_path}` t where lower(az_id) in ({ref_az_ids})")

spark.sql(f"""
insert into delta.`{config_refs_path}` (uuid, parent_az_id, az_id, last_delta_version, is_disabled)
    select uuid() uuid, parent_az_id, az_id, last_delta_version, is_disabled
    from tmp_config_refs
""")

In [7]:
display(spark.sql(f"select * from delta.`{config_path}` where config_name in ({configs})"))
display(spark.sql(f"DESCRIBE HISTORY delta.`{config_path}`"))

display(spark.sql(f"select * from delta.`{config_refs_path}` where az_id in ({ref_az_ids})"))
display(spark.sql(f"DESCRIBE HISTORY delta.`{config_refs_path}`"))